In [ ]:
!pip install mailgun

In [1]:
from google.colab import userdata

try:
  userdata.get("GOOGLE_API_KEY")
except:
    print("Enter your Google AI API key in the Secrets in Google Colab")

In [2]:
from google import genai

client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))

In [3]:
from google.genai import types

class Agent:
  def __init__(self, system_prompt="") -> None:
    self.system_prompt = system_prompt
    self.messages = []
    if self.system_prompt:
      self.messages.append(("system", system_prompt))

  def __call__(self, message):
    self.messages.append(("user", message))
    result = self.execute()
    self.messages.append(("assistant", result))
    return result

  def execute(self):
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=self.messages,
        config=types.GenerateContentConfig(
          temperature=0
        )
      )
    return response.text

In [60]:
prompt = """
You operate in a loop consisting of four stages: Thought, Action, PAUSE, and Observation. At the end of the loop, you provide an Answer.

Steps:

  Thought: Reflect on the question asked. Consider what action or calculation might provide the needed answer. (Show in the response)
  Action: Choose and perform one of the available actions. (Show in the response)

  PAUSE: Wait for the result of the action. (Do not show in the response)

  Observation: Once the action has been completed, you will receive the outcome. (Give in another user prompt)

  Answer: Finally, you’ll output the answer based on your observations. (Obviously show in the response)

Available Actions:

  find_email: Finds the email of the person in the Database using the first name
    e.g., find_email: John

  send_email: Sends the email using email, subject and body (For this action, you may have to figure out the subject and the body of the email depending on the user prompt) and REMEMBER: To give a valid JSON structure of the parameters as shown below and give the full JSON in one line as shown
    e.g., send_email: {
      "email": "john@gmail.com",
      "subject": "Test subject",
      "body": "Test body"
    }

Example Interaction:

User Prompt: "Send an email to John that I will meet him today at 5.00 PM in the office cafe"
Response:
  Thought: I need to send an email to John. So, first I have to find the email of John. To find this, I can use the 'find_email' action.
  Action: find_email: John

PAUSE

You will be called again with the result:
User Prompt:
"Observation: john@gmail.com"
Response:
  Thought: Now I have to send an email using the appropriate subject and body.
  Action: send_email: {"email": "john@gmail.com","subject": "Meeting at the office cafe","body": "Hi John, Lets meet today in our office cafe at 5.00 PM. \nBest Regards."}

PAUSE

User Prompt:
"Observation: Success. The email has been sent."
Response:
  Answer: I have succesfully sent the email to John.
""".strip()

In [71]:
import psycopg2
from mailgun.client import Client

def find_email(first_name: str) -> str:
  try:
    conn = psycopg2.connect(host=userdata.get("DB_HOST"), database=userdata.get("DB_DATABASE"), user=userdata.get("DB_USER"), password=userdata.get("DB_PASSWORD"))
    cursor = conn.cursor()
    cursor.execute("SELECT email FROM \"User\" WHERE first_name = %s", (first_name,))
    email = cursor.fetchone()
    if email:
        return email[0]
    else:
        return f"No email found for {first_name}"
  except Exception as err:
    return "Error: Unable to fetch the email from the database: {}".format(err)
  finally:
    cursor.close()
    conn.close()

def send_email(email: str, subject: str, body: str) -> str:
  try:
    client = Client(auth=("api", userdata.get("MAIL_API_KEY")))
    data = {
      "from": "no-reply@aiagent.com",
      "to": email,
      "subject": subject,
      "text": body
    }
    res = client.messages.create(data=data, domain=userdata.get("EMAIL_DOMAIN"))
    if res.status_code == 200:
      return "Success. The email has been sent."
    else:
      return "Failed to send the email."
  except Exception as err:
     return "Error: Failed to send the email: {}".format(err)

known_actions = {
    "find_email": find_email,
    "send_email": send_email
}

In [61]:
bot = Agent(prompt)

In [62]:
bot("Hi!")

'Thought: The user said "Hi!". This is a greeting. I should respond by acknowledging the greeting and asking how I can assist them.\nAction: None. I am waiting for a specific request or question from the user.Answer: Hello! How can I help you today?'

In [63]:
bot("Send an email to Abhi just saying hi and how he's doing")

"Thought: I need to send an email to Abhi. First, I need to find Abhi's email address. I can use the `find_email` action for this.\nAction: find_email: Abhi"

In [64]:
result = find_email("Abhi")

In [65]:
result

'kumarabhigna@gmail.com'

In [66]:
next_prompt = "Observation: {}".format(result)

In [67]:
bot(next_prompt)

'Thought: I have Abhi\'s email address. Now I need to send the email with the specified content. I will use the `send_email` action with the email address, a suitable subject, and the body "Hi Abhi, Just wanted to say hi and see how you\'re doing."\nAction: send_email: {"email": "kumarabhigna@gmail.com", "subject": "Checking In", "body": "Hi Abhi, Just wanted to say hi and see how you\'re doing."}'

In [24]:
import json

data = json.loads("""{\n  "email": "kumarabhigna@gmail.com",\n  "subject": "Checking in",\n  "body": "Hi Abhi, Just wanted to say hi and see how you\'re doing."\n}""")

In [25]:
data

{'email': 'kumarabhigna@gmail.com',
 'subject': 'Checking in',
 'body': "Hi Abhi, Just wanted to say hi and see how you're doing."}

In [27]:
result = send_email(**data)

In [28]:
result

'Success. The email has been sent.'

In [29]:
next_prompt = "Observation: {}".format(result)

In [30]:
bot(next_prompt)

'Answer: I have successfully sent the email to Abhi.'

In [58]:
import re

action_re = re.compile('^Action: (\w+): (.*)$', re.DOTALL)   # python regular expression to selection action

In [68]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input) if action != "send_email" else known_actions[action](**json.loads(action_input))
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [ ]:
query("Send an email to Abhi, just saying hi and how he's doing.")

In [76]:
query("Do you have email of Abhi")

Thought: The user is asking if I have the email of Abhi. To answer this, I need to use the `find_email` action to search for Abhi's email in the database.
Action: find_email: Abhi
 -- running find_email Abhi
Observation: kumarabhigna@gmail.com
Answer: Yes, I have Abhi's email. It is kumarabhigna@gmail.com.


In [77]:
query("Send an email to Abhi and Kumar, just saying hi and how they are doing.")

Thought: I need to send an email to Abhi. First, I need to find Abhi's email address. I can use the `find_email` action for this.
Action: find_email: Abhi
 -- running find_email Abhi
Observation: kumarabhigna@gmail.com
Thought: I have found Abhi's email address. Now I need to send an email to Abhi with the specified content. After sending the email to Abhi, I will need to find Kumar's email and send a similar email to him.
Action: send_email: {"email": "kumarabhigna@gmail.com", "subject": "Checking In", "body": "Hi Abhi, Just wanted to say hi and see how you are doing."}
 -- running send_email {"email": "kumarabhigna@gmail.com", "subject": "Checking In", "body": "Hi Abhi, Just wanted to say hi and see how you are doing."}
Observation: Success. The email has been sent.
Thought: I have successfully sent the email to Abhi. Now I need to send an email to Kumar. First, I need to find Kumar's email address. I can use the `find_email` action for this.
Action: find_email: Kumar
 -- running fin

In [79]:
query("Do you have email of Abhi and Kumar")

Thought: The user is asking if I have the email addresses of Abhi and Kumar. To determine this, I need to use the `find_email` action for each person. I will start with Abhi.
Action: find_email: Abhi
 -- running find_email Abhi
Observation: kumarabhigna@gmail.com
Thought: I have found the email for Abhi. Now I need to find the email for Kumar, as the original request was for both.
Action: find_email: Kumar
 -- running find_email Kumar
Observation: vaddepally.abhigna@gmail.com
Answer: Yes, I have the email for Abhi (kumarabhigna@gmail.com) and Kumar (vaddepally.abhigna@gmail.com).
